In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import datatable as dt
from datatable import f
from tqdm import tqdm
import os
import warnings
import gc

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder


from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
#import tensorflow_addons as tfa
from random import choices

from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import AUC
from keras.wrappers.scikit_learn import KerasClassifier

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format
warnings.filterwarnings("ignore")

from scipy import stats

In [ ]:
%%time

train = dt.fread('/kaggle/input/jane-street-market-prediction/train.csv')
train = train[ (f.date > 85 ) & (f.feature_43 > 1.0)  ,:].to_pandas()

print("data size:", train.shape)


features_columns = ["feature_%d" % i for i in range(130)]
columns_dtypes = {}


for column in features_columns:
    columns_dtypes[column] = "float32"


columns_dtypes["resp_1"] = "float32"
columns_dtypes["resp_2"] = "float32"
columns_dtypes["resp_3"] = "float32"
columns_dtypes["resp_4"] = "float32"
columns_dtypes["resp"] = "float32"
columns_dtypes["date"] = "int16"
columns_dtypes["ts_id"] = "int32"
columns_dtypes["weight"] = "float32"


train= train.astype(columns_dtypes)
train.info()



In [ ]:
features = [c for c in train.columns if 'feature' in c]
print(features)

In [ ]:
features_mean = []
for i in features:
    x = train[i].mean()  
    features_mean.append(x)
    train[i] = train[i].fillna(x)

In [ ]:

train['action'] = ((train['resp'].values) > 0).astype(int)
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']


X = train.loc[:, train.columns.str.contains('feature')]
#y_train = (train.loc[:, 'action'])

# resp_1 > 0 resp_2 > 0 ...
Y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T

In [ ]:
SEED=10001

def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)
    
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=[AUC(name='auc')],
    )

    return model



In [ ]:
epochs = 30
batch_size = 4096
hidden_units = [160, 160, 160]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-2

tf.random.set_seed(SEED)

clf = create_mlp(len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate)
clf.fit(X, Y, epochs=epochs, batch_size=batch_size, verbose=2)

In [ ]:
f_mean = np.mean(train[features[1:]].values,axis=0)

import janestreet
env = janestreet.make_env()
janestreet.make_env.__called__ = False

for (test_df, pred_df) in tqdm(env.iter_test()): 
    if test_df['weight'].item() > 0:
        
        x_tt = test_df.loc[:, features].values
        #if np.isnan(x_tt[:, 1:].sum()):
            # np.nan_to_num : Replace NaN with zero and infinity with large finite numbers
            # np.isnan : return a boolean list
            
        x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
            
        pred = np.median(clf(x_tt))
        pred_df.action = np.where(pred >= 0.50, 1, 0).astype(int)
        
    else:
        pred_df.action = 0
        
    env.predict(pred_df)